In [1]:
from langchain.llms import OpenAI
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.tools import StructuredTool
import json,os
from datetime import datetime
import pytz
from coro_agent.coro_agent import initialize_coro_agent, CoroAgentSuspend

memory = ConversationBufferMemory(memory_key='chat_history')

os.environ['OPENAI_API_KEY'] = '****'

llm = OpenAI(temperature=0)

def queryTime(action_input):
    '''
    查询此时此刻时间
    '''
    now = datetime.now()
    beijing_tz = pytz.timezone('Asia/Shanghai')
    beijing_time = now.astimezone(beijing_tz)
    return beijing_time.strftime("%Y-%m-%d %H:%M:%S")

def askUserCb(user_input: str):
    '''
    callback for askUser
    '''
    print(f'this is askUserCb {user_input}')
    return user_input

def askUser(action_input: str):
    '''
    当你无法确定可以调用哪个工具或者调用工具缺少必要参数的时候，你可以询问用户获取想要的答案，如果用户回答不满足，你可以重复提问，输入为:[你想了解的问题]
    '''    
    return CoroAgentSuspend(output=action_input, callback=StructuredTool.from_function(askUserCb))


def searchBill(action_input: str):
    '''
    用于查询指定日期账单, 查询参数不可编造，如果无法从用户提问中得知，需要向用户询问，需要的参数为json格式:
    {{
        "type": "账单类型:日账单|月账单", 
        "startDate": "开始日期, 如果是日账单需要精确到日,eg:20230627, 如果是月账单只需要精确到月,eg:202306", 
        "endDate": "结束日期,格式和startDate一致"
    }}
    '''
    params = json.loads(action_input)
    return '查询到账单如下：账单类型:{}\t日期:{}-{}\t 消费200元' .format(
        params['type'],
        params['startDate'],
        params['endDate'],        
    )

tools = [
    StructuredTool.from_function(searchBill),
    StructuredTool.from_function(askUser),
    StructuredTool.from_function(queryTime),
]


#agent_chain = initialize_coro_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)
# The agent `CONVERSATIONAL_REACT_DESCRIPTION`` have a aalignant bug， see: https://github.com/langchain-ai/langchain/issues/10311

agent_chain = initialize_coro_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, memory=memory)

r = agent_chain.run(input='我想查账单')
print(f'>>>>>>>>{r}')





> Entering new CoroAgentExecutor chain...
用户没有提供具体的账单类型和日期，我需要向用户询问这些信息。
Action: askUser
Action Input: 你想查询哪种类型的账单（日账单或月账单）？需要查询哪个日期范围的账单？
Observation: output='你想查询哪种类型的账单（日账单或月账单）？需要查询哪个日期范围的账单？' callback=StructuredTool(name='askUserCb', description='askUserCb(user_input: str) - callback for askUser', args_schema=<class 'pydantic.main.askUserCbSchemaSchema'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<function askUserCb at 0x7f6fa321a0d0>, coroutine=None)
Thought:
> Finished chain.
>>>>>>>>你想查询哪种类型的账单（日账单或月账单）？需要查询哪个日期范围的账单？


In [2]:
r = agent_chain.run(input='查询上个月的账单')
print(f'>>>>>>>>{r}')




> Entering new CoroAgentExecutor chain...
this is askUserCb 查询上个月的账单

Observation: 查询上个月的账单
Thought:用户想查询上个月的账单，但我还不知道现在的日期，我需要查询当前时间。
Action: queryTime
Action Input: 
Observation: 2023-09-26 17:53:06
Thought:现在是2023年9月，所以用户想查询的是2023年8月的账单，我现在可以查询账单了。
Action: searchBill
Action Input: {
    "type": "月账单", 
    "startDate": "202308", 
    "endDate": "202308"
}
Observation: 查询到账单如下：账单类型:月账单	日期:202308-202308	 消费200元
Thought:I now know the final answer
Final Answer: 查询到的账单如下：账单类型:月账单	日期:202308-202308	 消费200元

> Finished chain.
>>>>>>>>查询到的账单如下：账单类型:月账单	日期:202308-202308	 消费200元
